# RFZUpscale

**RFZUpscale** memanfaatkan **GPU Nvidia T4** di **Google Colab** untuk **meningkatkan kualitas gambar secara otomatis** dengan detail yang lebih tajam dan kejernihan yang lebih baik. Dengan dukungan GPU Nvidia T4.

In [ ]:
#@title 🚀 **Install Dependencies**
!pip install git+https://github.com/rizzbrew/Real-ESRGAN
!pip install -q huggingface_hub

import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger("huggingface_hub").setLevel(logging.ERROR)

from IPython.display import clear_output
clear_output()
print("✅ Semua dependensi telah diinstal.")

In [ ]:

#@title 📦 **Load Model, Upload, Enhance & Save**

from RealESRGAN import RealESRGAN
from PIL import Image
import numpy as np
import torch
import os
import sys
import tarfile
import shutil
from io import BytesIO
from google.colab import files
from google.colab import drive
from huggingface_hub import hf_hub_download

# ===== SETUP FOLDER =====
upload_folder = 'inputs'
result_folder = 'results'
weight_folder = 'weights'
os.makedirs(upload_folder, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)
os.makedirs(weight_folder, exist_ok=True)

# ===== DETEKSI CUDA =====
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('✅ Device:', device, flush=True)

# ===== MODEL & PATH =====
model_scale = "4"  #@param ["2", "4", "8"] {allow-input: false}
model_filename = f'RealESRGAN_x{model_scale}.pth'
model_path = os.path.join(weight_folder, model_filename)

# ===== CEK & UNDUH MODEL JIKA BELUM ADA =====
if not os.path.exists(model_path):
    try:
        print(f"🔄 Mengunduh model {model_filename}...", flush=True)
        hf_model_path = hf_hub_download(
            repo_id='ai-forever/Real-ESRGAN',
            filename=model_filename,
            local_dir=weight_folder,
        )
        shutil.move(hf_model_path, model_path)
        print(f"✅ Model berhasil diunduh ke {model_path}", flush=True)
    except Exception as e:
        print(f"❌ Gagal mengunduh model. Silakan unduh manual ke folder 'weights/'.", flush=True)
        print("Error:", e, flush=True)
        raise SystemExit

# ===== LOAD MODEL =====
try:
    model = RealESRGAN(device, scale=int(model_scale))
    model.load_weights(model_path)
    print(f"✅ Model x{model_scale} berhasil dimuat.\n", flush=True)
except Exception as e:
    print("❌ Gagal memuat model.", flush=True)
    print("Error detail:", e, flush=True)
    raise SystemExit

# ===== FORMAT TAR SUPPORT =====
IMAGE_FORMATS = ('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')

def image_to_tar_format(img, image_name):
    buff = BytesIO()
    if '.png' in image_name.lower():
        img = img.convert('RGBA')
        img.save(buff, format='PNG')
    else:
        img.save(buff, format='JPEG')
    buff.seek(0)
    fp = BytesIO(buff.read())
    img_tar_info = tarfile.TarInfo(name=image_name)
    img_tar_info.size = len(fp.getvalue())
    fp.seek(0)
    return img_tar_info, fp

def process_tar(path_to_tar):
    with tarfile.open(path_to_tar, mode='r:*') as processing_tar:
        result_tar_path = os.path.join(result_folder, os.path.basename(path_to_tar))
        with tarfile.open(result_tar_path, 'w') as save_tar:
            for member in processing_tar:
                if not member.name.lower().endswith(IMAGE_FORMATS):
                    continue
                try:
                    img_bytes = BytesIO(processing_tar.extractfile(member.name).read())
                    img_lr = Image.open(img_bytes).convert('RGB')
                except Exception:
                    print(f'⚠️ Tidak bisa membuka file {member.name}, dilewati.', flush=True)
                    continue

                try:
                    img_sr = model.predict(np.array(img_lr))
                except Exception as e:
                    print(f"❌ Gagal memproses gambar {member.name}, dilewati. Error: {e}", flush=True)
                    continue

                img_tar_info, fp = image_to_tar_format(img_sr, member.name)
                save_tar.addfile(img_tar_info, fp)

    print(f'✅ Selesai! File archive disimpan di {result_tar_path}', flush=True)

def process_input(filename):
    if tarfile.is_tarfile(filename):
        process_tar(filename)
    else:
        result_image_path = os.path.join(result_folder, '[HD] ' + os.path.basename(filename))
        try:
            image = Image.open(filename).convert('RGB')
            sr_image = model.predict(np.array(image))
            sr_image.save(result_image_path)
            print(f'✅ Selesai memproses {filename}\n', flush=True)
        except Exception as e:
            print(f"❌ Gagal memproses gambar {filename}. Error: {e}", flush=True)

        if os.path.exists(filename):
            os.remove(filename)

# ===== MOUNT GOOGLE DRIVE DENGAN ERROR HANDLING =====
def mount_drive():
    try:
        print("⚡ Mounting Google Drive...", flush=True)
        drive.mount('/content/drive')
        print("✅ Google Drive berhasil dimount.", flush=True)
        return True
    except Exception:
        print("❌ Gagal otorisasi Google Drive.", flush=True)
        return False

# ===== MAIN FUNCTION =====
def main():
    try:
        print('📁 Pilih gambar/tar file untuk di-enhance:\n', flush=True)
        uploaded = files.upload()

        if not uploaded:
            print("❌ Tidak ada file yang diunggah. Proses dibatalkan.", flush=True)
            return

        for filename in uploaded.keys():
            print(f'✨ Memperbesar dan meningkatkan kualitas: {filename}', flush=True)
            process_input(filename)

        while True:
            pilihan = input(
                "\nPilih opsi simpan hasil:\n"
                "1. Simpan ke Google Drive (folder RFZUpscale)\n"
                "2. Download ZIP hasil\n"
                "Masukkan nomor pilihan (1/2) atau tekan Enter untuk default (1): "
            ).strip()

            if pilihan == '':
                pilihan = '1'

            if pilihan not in ('1', '2'):
                print("⚠️ Masukkan angka 1 atau 2 saja!", flush=True)
            else:
                break

        if pilihan == '1':
            if mount_drive():
                drive_save_folder = '/content/drive/MyDrive/RFZUpscale'
                os.makedirs(drive_save_folder, exist_ok=True)
                for f in os.listdir(result_folder):
                    shutil.copy(os.path.join(result_folder, f), drive_save_folder)
                print(f"✅ Hasil disimpan di Google Drive:\n{drive_save_folder}", flush=True)
            else:
                print("❌ Penyimpanan ke Google Drive dibatalkan.", flush=True)

        elif pilihan == '2':
            if os.path.exists(result_folder):
                shutil.make_archive('download', 'zip', result_folder)
                files.download('download.zip')
                print("✅ File ZIP siap diunduh.", flush=True)
            else:
                print("⚠️ Folder hasil tidak ditemukan, tidak ada file ZIP yang dibuat.", flush=True)

    except KeyboardInterrupt:
        print('\n❌ Upload dibatalkan oleh pengguna.', flush=True)
    finally:
        if os.path.exists(upload_folder):
            shutil.rmtree(upload_folder)
        if os.path.exists(result_folder):
            shutil.rmtree(result_folder)

if __name__ == '__main__':
    main()